We can use the world data to to make a time series heatMap for the prcentage population being affected throughout the world to bring forth the visualization, how covid-19 is making rounds around the world

In [30]:
import pandas as pd
from pathlib import Path
import numpy as np
import os
from tqdm.notebook import tqdm
import time
from geopy.geocoders import Nominatim
import random
from pprint import pprint
import folium
import folium.plugins as plugins

In [31]:
data_path = Path(os.path.join(os.getcwd(),'covid19-global-forecasting-week-3'))
train = pd.read_csv(os.path.join(data_path , 'train.csv'), parse_dates=['Date'])
test = pd.read_csv(os.path.join(data_path , 'test.csv'), parse_dates=['Date'])
# Load the population data into lookup dicts
pop_info = pd.read_csv(os.path.join(os.getcwd(),'covid19-population-data','population_data.csv'))

In [32]:
train.shape

(22950, 6)

In [33]:
train = train.sort_values(by = 'Date', ascending=True)
time_index = [str(k).split('T')[0] for k in train['Date'].unique()]
train = train.sort_values(by = 'Date', ascending=False)

In [34]:
def merged_location(row):
    if str(row['Province_State']) =="nan":
        return row['Country_Region']
    else:
        return row['Province_State'] + " " +row['Country_Region']

In [35]:
train['Location'] = train.apply(lambda row: merged_location(row), axis=1)

In [36]:
train['NewConfirmedCases'] = 0

In [37]:
#%%pixie_debugger
for loc in tqdm(train['Location'].unique()):
    df = train[train['Location'] == loc]
    l = len(df)
    df = df.sort_values(by = 'Date', ascending=True)
    for i in df.index[1:]:
        train.loc[i,'NewConfirmedCases'] = train.loc[i,'ConfirmedCases']-train.loc[i-1,'ConfirmedCases'] 

In [ ]:
def lat_long(location):
    time.sleep(random.choice(range(1, 4,1)))
    try:
        from geopy.geocoders import Nominatim
        geolocator = Nominatim(timeout=10)
        geo_location  = geolocator.geocode(location)
#         print("{}- lat - {} - long {}".format(location, geo_location.latitude, geo_location.longitude))
        return geo_location.latitude, geo_location.longitude
    except Exception as e:
        print(str(e))
        print(location)
        return np.nan, np.nan
        pass

In [ ]:
loc_dec = {}
for loc in tqdm(train['Location'].unique()):
    loc_dec[loc] ={}
    lat, long = lat_long(loc)
    if lat == np.nan:
        pass
    else:
        loc_dec[loc]['latitude'] = lat
        loc_dec[loc]['longitude'] = long

In [ ]:
# 51.510384, -0.1570916
# 51.5088472, -0.1449478
# 49.4462088, -2.9028271
# 51.5199932, -0.1590354
loc_dec['British Virgin Islands United Kingdom']['latitude'] = 51.510384
loc_dec['British Virgin Islands United Kingdom']['longitude'] = -0.1570916
loc_dec['Cayman Islands United Kingdom']['latitude'] = 51.5088472
loc_dec['Cayman Islands United Kingdom']['longitude'] = -0.1449478
loc_dec['Channel Islands United Kingdom']['latitude'] = 49.4462088
loc_dec['Channel Islands United Kingdom']['longitude'] = -2.9028271
loc_dec['Turks and Caicos Islands United Kingdom']['latitude'] = 51.5199932
loc_dec['Turks and Caicos Islands United Kingdom']['longitude'] = -0.1590354

In [ ]:
pprint(loc_dec)

In [38]:
import json 
  
# Opening JSON file 
f = open('world_coordinate.json',) 
  
# returns JSON object as  
# a dictionary 
loc_dec = json.load(f) 

In [39]:
def lat(row):
    return loc_dec[row['Location']]['latitude']
def long(row):
    return loc_dec[row['Location']]['longitude']

In [40]:
train['Latitude'] = train.apply(lambda row: lat(row), axis=1)
train['Longitude'] = train.apply(lambda row: long(row), axis=1)

In [41]:
country_pop = pop_info.query('Type == "Country/Region"')
province_pop = pop_info.query('Type == "Province/State"')
country_lookup = dict(zip(country_pop['Name'], country_pop['Population']))
province_lookup = dict(zip(province_pop['Name'], province_pop['Population']))

In [42]:
def get_population(row):
    try:
        population = province_lookup[row['Province_State']]
    except:
        population = country_lookup[row['Country_Region']]
    return population

In [43]:
train['Population'] = train.apply(lambda row: get_population(row), axis=1)

In [44]:
def weight(row):
    weight = (row['NewConfirmedCases'] / row['Population']) #+ 0.5 * (row['Fatalities'] / row['Population'])
    if weight < 0:
        return 0
    else:
        return weight

In [45]:
train['Weight'] = train.apply(lambda row: weight(row), axis=1)

In [46]:
train

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,Location,NewConfirmedCases,Latitude,Longitude,Population,Weight
22949,32710,NaN,Zimbabwe,2020-04-05,9.0,1.0,Zimbabwe,0.0,-18.455496,29.746841,16530000,0.000000e+00
374,503,NaN,Angola,2020-04-05,14.0,2.0,Angola,4.0,-11.877577,17.569124,32866272,1.217053e-07
2624,3713,NaN,Burma,2020-04-05,21.0,1.0,Burma,0.0,17.175050,95.999965,53370000,0.000000e+00
1349,1894,NaN,Azerbaijan,2020-04-05,584.0,7.0,Azerbaijan,63.0,40.393629,47.787251,10139177,6.213522e-06
1199,1680,Western Australia,Australia,2020-04-05,453.0,3.0,Western Australia Australia,17.0,-25.230300,121.018725,2589000,6.566242e-06
...,...,...,...,...,...,...,...,...,...,...,...,...
10950,15623,NaN,Jamaica,2020-01-22,0.0,0.0,Jamaica,0.0,18.115296,-77.159845,2961167,0.000000e+00
13200,18833,NaN,Morocco,2020-01-22,0.0,0.0,Morocco,0.0,31.172821,-7.336248,36910560,0.000000e+00
2400,3425,NaN,Bulgaria,2020-01-22,0.0,0.0,Bulgaria,0.0,42.607397,25.485662,6948445,0.000000e+00
18150,25895,Georgia,US,2020-01-22,0.0,0.0,Georgia US,0.0,32.329381,-83.113737,10520000,0.000000e+00


In [47]:
len(train['Date'].unique())

75

In [48]:
len(train[train['Date'] == '2020-01-23'])

306

In [49]:
data = np.ndarray(shape=(75,306, 3), dtype=float)

In [50]:
data = np.ndarray(shape=(75,306, 3), dtype=float)
for i, date in enumerate(train['Date'].unique()):
    data[i] = train[train['Date'] == date][['Latitude','Longitude','Weight']].values

In [51]:
train[train['Date'] == '2020-04-05']

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,Location,NewConfirmedCases,Latitude,Longitude,Population,Weight
22949,32710,NaN,Zimbabwe,2020-04-05,9.0,1.0,Zimbabwe,0.0,-18.455496,29.746841,16530000,0.000000e+00
374,503,NaN,Angola,2020-04-05,14.0,2.0,Angola,4.0,-11.877577,17.569124,32866272,1.217053e-07
2624,3713,NaN,Burma,2020-04-05,21.0,1.0,Burma,0.0,17.175050,95.999965,53370000,0.000000e+00
1349,1894,NaN,Azerbaijan,2020-04-05,584.0,7.0,Azerbaijan,63.0,40.393629,47.787251,10139177,6.213522e-06
1199,1680,Western Australia,Australia,2020-04-05,453.0,3.0,Western Australia Australia,17.0,-25.230300,121.018725,2589000,6.566242e-06
...,...,...,...,...,...,...,...,...,...,...,...,...
22124,31533,Gibraltar,United Kingdom,2020-04-05,103.0,0.0,Gibraltar United Kingdom,5.0,52.141155,1.200945,34571,1.446299e-04
22199,31640,Isle of Man,United Kingdom,2020-04-05,127.0,1.0,Isle of Man United Kingdom,1.0,54.235817,-4.514599,84287,1.186423e-05
22274,31747,Montserrat,United Kingdom,2020-04-05,6.0,0.0,Montserrat United Kingdom,0.0,51.728386,-2.208550,5900,0.000000e+00
22349,31854,Turks and Caicos Islands,United Kingdom,2020-04-05,5.0,1.0,Turks and Caicos Islands United Kingdom,0.0,51.519993,-0.159035,35446,0.000000e+00


In [52]:
min(train['Weight'])

0.0

In [53]:
max(train['Weight'])

0.026259946949602122

In [54]:
m = folium.Map([48., 5.], tiles='stamentoner', zoom_start=4)

hm = plugins.HeatMapWithTime(
    data.tolist(),
    index=time_index,
    auto_play=False#,
#     max_opacity=0.3
)

hm.add_to(m)

m

In [71]:
train.to_csv('time_series_world.csv')

In [72]:
m.save('Time Series Covid 19.html')